[Support Vector classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)  
[Logistic Classifier](https://scikitlearn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

c2 = SVC(kernel='linear')
c3 = LogisticRegression(C=10,solver='lbfgs')
model = dict([("Bayes",c1),("SVC",c2),("Logit",c3)])

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [5]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

In [15]:
answer_five()

0.9416243654822335

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [16]:
def answer_six():
    al = [0,0]
    for i in [0,1]:
        samples = spam_data[ spam_data.target==i]
        l = samples.text.apply(len)
        al[i]=l.mean()
    return tuple(al)

In [17]:
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [18]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [19]:
from sklearn.svm import SVC

def answer_seven():
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_traint = vect.transform(X_train)
    len_doc = X_train.apply(len).values
    X_traint_new = add_feature(X_traint,len_doc)
    
    clf = SVC(C=10000)
    clf.fit(X_traint_new,y_train)
    X_testt = vect.transform(X_test)
    len_tdocs = X_test.apply(len).values
    X_test_new = add_feature(X_testt,len_tdocs)
    y_pred = clf.predict(X_test_new)
    
    return roc_auc_score(y_test,y_pred)

In [20]:
answer_seven()

/home/cesarliz10/.local/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.9581366823421557

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [21]:
def answer_eight():
    av_ndig = [0,0]
    count_digits = lambda text: sum([ x.isdigit() for x in  list(text)])
    for i in [0,1]:
        samples = spam_data[ spam_data.target==i]
        d = samples.text.apply(count_digits)
        av_ndig[i]=d.mean()
    
    return tuple(av_ndig)

In [22]:
answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [23]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    vect = TfidfVectorizer(min_df=5,ngram_range=(1,3)).fit(X_train)
    X_traint = vect.transform(X_train)
    len_doc = X_train.apply(len).values
    count_digits = lambda text: sum([ x.isdigit() for x in  list(text)])
    n_digs = X_train.apply(count_digits).values
    X_traint_new = add_feature(X_traint,[len_doc,n_digs])
    clf = LogisticRegression(C=100)
    clf.fit(X_traint_new,y_train)
    
    X_testt = vect.transform(X_test)
    len_doc = X_test.apply(len).values
    n_digs = X_test.apply(count_digits).values
    X_testt_new = add_feature(X_testt,[len_doc,n_digs])
    y_pred = clf.predict(X_testt_new)
    
    return roc_auc_score(y_test,y_pred)

In [24]:
answer_nine()

/home/cesarliz10/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9653328353394565

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [25]:
def answer_ten():
    av_nchar = [0,0]
    for i in [0,1]:
        samples = spam_data[ spam_data.target==i]
        av_nchar[i]=samples["text"].str.count(r'\W').mean()
    return tuple(av_nchar)


In [26]:
answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [27]:
def answer_eleven():
    vect = CountVectorizer(analyzer='char_wb',min_df=5, ngram_range=(2,5))
    vect.fit(X_train)
    
    doc_len = X_train.str.len().values
    doc_dig = X_train.str.count(r'\d').values
    doc_nwc = X_train.str.count(r'\W').values
    X_train_new = add_feature(vect.transform(X_train),[doc_len,doc_dig,doc_nwc])

    clf = LogisticRegression(C=100).fit(X_train_new,y_train)
    
    doc_len = X_test.str.len().values
    doc_dig = X_test.str.count(r'\d').values
    doc_nwc = X_test.str.count(r'\W').values
    X_test_new = add_feature(vect.transform(X_test),[doc_len,doc_dig,doc_nwc])
    
    y_pred = clf.predict(X_test_new)

    auc_score = roc_auc_score(y_test,y_pred)
    
    vv = vect.vocabulary_
    for i,j in zip([16314,16315,16316],['length_of_doc', 'digit_count', 'non_word_char_count']):
        vv[j] = i 
    extended_voc = dict(zip(vv.values(),vv.keys()))
    idx_sorted = clf.coef_[0].argsort()
    
    scl= [extended_voc[i] for i in idx_sorted[:10]]
    lcl= [extended_voc[i] for i in idx_sorted[-10:]]
    
    return (auc_score,scl,lcl)
    

In [28]:
answer_eleven()

/home/cesarliz10/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(0.9788593110707434,
 ['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'],
 ['ar', 'ww', ' x', 'mob', ' ch', 'xt', 'co', 'ia', 'ne', 'digit_count'])